In [ ]:
from IPython import get_ipython
from IPython.display import display

get_ipython().system('pip install --quiet --upgrade langchain langchain-community langchain-chroma langchain_huggingface ragas pypdf')
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()
os.environ["GROQ_API_KEY"] = getpass.getpass()

import langchain
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import LLMChain
import json
from ragas import evaluate
from datasets import Dataset

In [4]:
from langchain_groq import ChatGroq

In [61]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from ragas.metrics import faithfulness as faithfulness_metric
from ragas.metrics import answer_relevancy as answer_relevancy_metric

In [5]:
file_path = "/content/MergeResult_2024_09_25_07_16_45.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

165


In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)


In [ ]:
embeddings = HuggingFaceEmbeddings(model_name='intfloat/multilingual-e5-large')

In [8]:
vectorstore = Chroma.from_documents( splits, embeddings )

In [40]:
llm = ChatGroq(temperature=0,  model_name="mixtral-8x7b-32768")

In [69]:
prompt = """
Answer the question based only on the following context:
{context}
Question: {question}

the output should be the question then the answer in a json format.
"""

prompt = ChatPromptTemplate.from_template(prompt)

In [70]:
model = LLMChain(llm=llm, prompt=prompt)

In [31]:
retriever = vectorstore.as_retriever()

In [71]:
rag_chain = (
    {"context": retriever , "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [92]:
respone = rag_chain.invoke("What is usability?")

In [73]:
respone

'{\n"question": "What is usability?",\n"answer": "Usability is a measure of the effectiveness, efficiency, and satisfaction with which specified users can achieve specified goals in a particular environment."\n}'

In [63]:
import openai
openai.api_key = ''

In [ ]:
data_sample = [
    {
        "question": "What is a native application?",
        "answer": "A native application is built for use on a particular device and its operating system. It has the ability to use device-specific hardware and software",
        "retrieved_contexts": ["A native application refers to software built for use on a particular device and its operating system. It allows access to device-specific hardware and software features such as the camera, GPS, etc."] # Changed to a list of strings
    },
    {
        "question": "What is a web application?",
        "answer": "A web application is an application that the user does not download but accesses via a web browser over a network",
        "retrieved_contexts": ["Web applications are accessed via browsers like Chrome, Safari, or Firefox over a network and are not installed on the device. These applications use HTML, CSS, and JavaScript to deliver content and functionality."] # Changed to a list of strings
    }
]

dataset = Dataset.from_list(data_sample)
score = evaluate(dataset, [answer_relevancy_metric, faithfulness_metric])
score_df = score.to_pandas()

print(score_df)

In [98]:
j_llm = ChatGroq(model="llama3-8b-8192")

In [101]:
def judge_eval(question, answer):

  judge_prompt = f"""
  You are an expert evaluator. Given the following question and the answer generated by an LLM.
  Is the answer relevant to the question? and does the model hallucination?

  Question: {question}
  Answer: {answer}

  Evaluation:
  """

  evaluation = j_llm.invoke(judge_prompt)

  return {"evaluation": evaluation}




In [93]:
respone = json.loads(respone)

In [102]:
result = judge_eval(respone['question'], respone['answer'])
print(result['evaluation'])

content='Evaluation:\n\nRelevance to the question: **Highly Relevant**\n\nThe answer provides a clear and concise definition of a native application, which aligns perfectly with the question. The answer covers the key aspects of native applications, including their development for a specific platform, installation on a device, and access to device-specific features.\n\nHallucination: **No Hallucination**\n\nThe answer does not contain any information that is not supported by the provided definition. The language used is clear and accurate, and the answer does not introduce any unrelated concepts or details that are not relevant to the question. The answer stays focused on providing a comprehensive definition of a native application, without adding any fictional or invented information.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 140, 'prompt_tokens': 155, 'total_tokens': 295, 'completion_time': 0.116666667, 'prompt_time': 0.026097791, 'queue_time': 0.0

In [103]:
respone = rag_chain.invoke("What is a native application?")
respone = json.loads(respone)
print(respone['answer'])
result = judge_eval(respone['question'], respone['answer'])
print(result['evaluation'])

A native application is a software program that is developed for a specific platform and installed on a computing device. In the context of mobile web applications, it is used to refer to any application written to work on a specific device platform, with access to the full functionality of the device, including databases and hardware features, and the ability to use device-specific hardware and software. Native applications can provide a higher level of security and personalization compared to other types of applications.
content="**Relevance to the question:** 9/10\n\nThe answer provides a clear and concise definition of a native application, which is directly relevant to the question. The answer also provides additional information about the characteristics of native applications, such as their ability to access device functionality and provide higher levels of security and personalization. The only reason I wouldn't give it a perfect score is that the answer doesn't explicitly stat

In [104]:
respone = rag_chain.invoke("What is a web application?")
respone = json.loads(respone)
print(respone['answer'])
result =judge_eval(respone['question'], respone['answer'])
print(result['evaluation'])

A web application (WB) is an application that the user does not download and instead accesses via a web browser over a network. It provides functionality, such as bank account access or YouTube video viewing, and can be written in JavaScript, CSS, and standard HTML for universal use across various browsers. Web applications can use a single code base because they are not designed around a specific device, and are fast and simple to build but not as quick as native applications. They are developed to be generic across multiple systems and differ from mobile native applications in that they use web technologies and are not limited to the underlying platform for deployment.
content='**Relevance to the question:** The answer is highly relevant to the question "What is a web application?" It provides a clear and concise definition of a web application, explaining that it is an application accessed via a web browser, and describes its characteristics, advantages, and differences from native 